In [2]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### vectorStore 지원 검색기 는 vector store를 사용하여 문서를 검색하는 retriever입니다.
**vectorStore** 를 만들고 `as_retriever()`를 호출하게 되면 vectorStore 에 종속적인 retriever를 얻을수 있습니다.<br>
vectorStore 지원 검색기 라는 것은 우리가 어떤 vectorStore를 선택했는가에 따라서 달라질 수 있습니다.<br>
Chroma DB를 사용한다면 Chroma DB 기반의 검색기를 가져오는 것이고 <br> 
FAISS DB를 사용한다면 FAISS DB의 as_retriever()를 호출해서 가져오면 FAISS DB에 종속 되어 있는 검색기를 가져옵니다.<br>

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [4]:
# TextLoader를 사용하여 파일을 로드합니다.
loader = TextLoader("./data/appendix-keywords.txt", encoding='utf-8')

In [5]:
# 문서를 로드합니다.
documents = loader.load()

In [6]:
# 문자 기반으로 텍스트를 분할하는 CharacterTextSplitter를 생성합니다. 
# 청크 크기는 300, 청크 간 중복은 없습니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

In [7]:
# 로드된 문서를 분할합니다.
split_docs = text_splitter.split_documents(documents)

In [8]:
# OpenAI 임베딩을 생성.
embeddings = OpenAIEmbeddings(api_key=key)

In [9]:
# 분할된 텍스트와 임베딩을 사용하여 FAISS 벡터 데이터베이스를 생성
db = FAISS.from_documents(split_docs, embeddings)

## `as_retriever()` 메소드는 VectorStore객체를 기반으로 VectorStoreRetriever를 반환합니다. <br>
`as_retriever()`를 호출해서 검색기를 가져올 수 있습니다. <br><br>
`as_retriever()`를 호출해서 검색기를 가져 올 때 설정 값 들을 지정할 수 있습니다. <br>

**매개변수(parameters)**
- `**kwargs`: 검색 함수에 전달할 키워드 인자
    - `search_type`: 검색 유형 ("similarity", "mmr", "similarity_score_threshold")
        - 기본값으로 similarity가 적용되어있습니다.
        -  mmr, similarity_score_threshold 로 변경할 수 있습니다.
            - similarity(검색된 결과에서 유사도만 보는 것)
            - mmr(검색된 결과를 가지고 다양성을 적용해서 결과에 반영해주는것)
            - similarity_score_threshold
                -  (유사도 검색을 하는데 문턱값을 줘서 유사도가 몇 % 이상인 문서들만 걸러내는 것)
    - `search_kwargs`: 세부적인 추가 옵션
    - `k`: 문서의 검색에서 몇 건의 문서를 반환할 것인지 설정. 반환 되는 문서의 수 (기본값: 4)
    - `score_threshold`: similarity_score_threshold 검색의 최소 유사도 임계값
        - similarity_score_threshold를 적용했을 때 작동한다.
        - score_threshold 값을 만약 0.8로 지정했다면 0.8 이상인 문서만 검색결과에 걸리게 된다.
    - `fetch_k`: MMMR 알고리즘에 전달할 문서 수 (기본값: 20)
        - 초기에 검색할 문서의 개수이다. 
        - mmr 알고리즘을 적용했을 때만 된다.
        - mmr 알고리즘을 지정하면은 문서를 검색해서 먼저 fetch_k 만큼의 문서를 검색하게 된다.
    - `lambda_mult`: MMR 결과의 다양성 조절 (0-1 사이, 기본값: 0.5)
    - `filter`: 문서 메타데이터 기반 필터링
        - 문서의 소스를 여러 군데에서 넣었을 경우에 a,b,c 문서가 있다면 필터에 소스를 걸어서 해당 문서 소스 에서만 검색되도록 필터링

In [10]:
# 데이터베이스를 검색기로 사용하기 위해 retriever 변수에 할당
retriever = db.as_retriever()

### Retriever의 invoke() <br>
`invoke` 메소드는 Retriever의 주요 진입점으로, 관련 문서를 검색하는 데 사용됩니다. <br>
이 메소드는 동기적으로 Retriever를 호출하여 주어진 쿼리에 대한 관련 문서를 반환합니다. <br><br>

**매개변수(parameters)**
- `input`: 검색 쿼리 문자열
- `config`: Retriever 구성 (Optional[RunnableConfig])
- `**kwargs`: Retriever에 전달할 추가 인자

In [11]:
# 관련 문서를 검색
docs = retriever.invoke("임베딩(Embedding)은 무엇인가요?")  # k가 기본값이라서 4개의 문서가 검색. 유사도 기반으로 내림차순.

for doc in docs:
    print(doc.page_content)
    print("=========================================================")

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token
정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)
Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding
정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec


### MMR(Maximal Marginal Relevance) <br>
다양성을 같이 본다. <br><br>


**매개변수(parameters)**

- `search_type` 매개변수를 "mmr"로 설정해서 MMR(Maximal Marginal Relevance) 검색 알고리즘 사용.
- `k`: 반환할 문서 수 (기본값: 4)
- `fetch_k`: MMMR 알고리즘에 전달할 문서 수 (기본값: 20)
- `lambda_mult`: MMR 결과의 다양성 조절 (0~1, 기본값: 0.5, 0: 유사도 점수만 고려, 1: 다양성만 고려)

In [12]:
# MMR(Maximal Marginal Relevance) 검색 유형을 지정
# 처음에는 10개의 문서를 검색한 다음에 10개의 문서안에서 다양성(0.6)을 보겠다.
# 결과를 댜양하게 나올 수 있도록 조정했다. 그 다양성으로 고려해서 2개를 뽑는다.

# k: 2 (최종적으로 검색되는 문서 2건)
# fetch_k : 10 (처음에는 10개의 문서를 검색)
# lambda_mult : 0.6 (다양성)
retriever = db.as_retriever(
    search_type="mmr", search_kwargs={"k": 2, "fetch_k": 10, "lambda_mult": 0.6}
)

In [13]:
docs = retriever.invoke("임베딩(Embedding)은 무엇인가요?")

for doc in docs:
    print(doc.page_content)
    print("=========================================================")

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token
Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding


###  유사도 점수 임계값 검색(similarity_score_threshold)
`similarity_score_threshold`을 설정하고 score_threshold 값을 설정해서 사용.

In [14]:
retriever = db.as_retriever(    
    search_type="similarity_score_threshold",       # 검색 유형을 "similarity_score_threshold 으로 설정    
    search_kwargs={"score_threshold": 0.8},         # 임계값 설정. 검색을 할 때 유사도 기반으로 0.8 이상인 문서들만 검색한다. 이때 k는 의미 없다.
)

In [15]:
for doc in retriever.invoke("Word2Vec 은 무엇인가요?"):     # 1개의 문서가 출력. k는 의미 없다. 0.8 이상인것이 1개라서. 
    print(doc.page_content)
    print("=========================================================")

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)


### top_k 설정
- `k` 를 의미한다. 상위 몇 개의 문서를 검색결과에 포함시킬 것인가를 top_k라고 한다.

In [26]:
# k 설정
retriever = db.as_retriever(search_kwargs={"k": 1})     # 1개의 문서만 반환하라.

In [17]:
docs = retriever.invoke("임베딩(Embedding)은 무엇인가요?")

# 관련 문서를 검색
for doc in docs:
    print(doc.page_content)
    print("=========================================================")

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token


### 동적 설정(Configurable)
`as_retriever()` 해서 검색기를 만드는 단계에서 파라미터를 지정하지 않고 invoke() 할때 변경할 수 있다 <br><br>

- 검색 설정을 동적으로 조정하기 위해 **ConfigurableField** 를 사용.
- **ConfigurableField** 는 검색 매개변수의 고유 식별자, 이름, 설명을 설정하는 역할을 한다.
- 검색 설정을 조정하기 위해 **config** 매개변수를 사용하여 검색 설정을 지정한다.
- 검색 설정은 config 매개변수에 전달된 딕셔너리의 **configurable** 키에 저장된다.
- 검색 설정은 검색 쿼리와 함께 전달되며, 검색 쿼리에 따라 동적으로 조정됨.

In [18]:
from langchain_core.runnables import ConfigurableField

In [19]:
# k 설정
retriever = db.as_retriever(search_kwargs={"k": 1}).configurable_fields(  
    search_type=ConfigurableField(
        id="search_type",
        name="Search Type",
        description="The search type to use",
    ),
    search_kwargs=ConfigurableField(        
        id="search_kwargs",                         # 검색 매개변수의 아이디 설정        
        name="Search Kwargs",                       # 검색 매개변수의 이름 설정        
        description="The search kwargs to use",     # 검색 매개변수에 대한 설명을 작성
    ),
)

In [20]:
# 검색 설정을 지정. Faiss 검색에서 k=3로 설정하여 가장 유사한 문서 3개를 반환
# search_kwargs는 위에서 지정한 id이다.
config = {"configurable": {"search_kwargs": {"k": 3}}}

# 설정값을 동적으로 넣을 수 있다. 
docs = retriever.invoke("임베딩(Embedding)은 무엇인가요?", config=config)

In [21]:
for doc in docs:
    print(doc.page_content)
    print("=========================================================")

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token
정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)
Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding


In [22]:
# 검색 설정을 지정. score_threshold 0.8 이상의 점수를 가진 문서만 반환
config = {
    "configurable": {
        "search_type": "similarity_score_threshold",
        "search_kwargs": {
            "score_threshold": 0.8,
        },
    }
}

In [23]:
# 설정값을 동적으로 넣을 수 있다. 
docs = retriever.invoke("Word2Vec 은 무엇인가요?", config=config)

# 관련 문서를 검색
for doc in docs:
    print(doc.page_content)
    print("=========================================================")

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)


In [24]:
# 검색 설정을 지정. mmr 검색 설정.
config = {
    "configurable": {
        "search_type": "mmr",
        "search_kwargs": {"k": 2, "fetch_k": 10, "lambda_mult": 0.6},
    }
}

In [25]:
docs = retriever.invoke("Word2Vec 은 무엇인가요?", config=config)

# 관련 문서를 검색
for doc in docs:
    print(doc.page_content)
    print("=========================================================")

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)
정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec
